In [1]:
#!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [2]:
import tiktoken
import torch
import torch.nn as nn
from torch.nn import functional as F

In [3]:
with open("./data/input.txt", "r", encoding="utf-8") as f:
    text = f.read()


In [4]:
print(f"length of dataset {len(text)}")

length of dataset 1115394


In [5]:
print(text[:300])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us


In [6]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [7]:
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [8]:
# map chars to integers
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [9]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [10]:
n = int(0.9*len(data))
train_data = data[:n]
test_data = data[n:]

In [11]:
# time dimension
block_size=8
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"context {context} produces target {target}")

context tensor([18]) produces target 47
context tensor([18, 47]) produces target 56
context tensor([18, 47, 56]) produces target 57
context tensor([18, 47, 56, 57]) produces target 58
context tensor([18, 47, 56, 57, 58]) produces target 1
context tensor([18, 47, 56, 57, 58,  1]) produces target 15
context tensor([18, 47, 56, 57, 58,  1, 15]) produces target 47
context tensor([18, 47, 56, 57, 58,  1, 15, 47]) produces target 58


In [12]:
# batch dimension
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split):
    data = train_data if split=='train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb, yb = get_batch('train')
print('inputs')
print(xb.shape)
print(xb)
print('targets')
print(yb.shape)
print(yb)


for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f'input {context.tolist()} gives target {target}')

inputs
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
input [24] gives target 43
input [24, 43] gives target 58
input [24, 43, 58] gives target 5
input [24, 43, 58, 5] gives target 57
input [24, 43, 58, 5, 57] gives target 1
input [24, 43, 58, 5, 57, 1] gives target 46
input [24, 43, 58, 5, 57, 1, 46] gives target 43
input [24, 43, 58, 5, 57, 1, 46, 43] gives target 39
input [44] gives target 53
input [44, 53] gives target 56
input [44, 53, 56] gives target 1
input [44, 53, 56, 1] gives target 58
input [44, 53, 56, 1, 58] gives target 46
input [44, 53, 56, 1, 58, 46] gives target 39
input [44, 53, 56, 1, 58, 46, 39] gives target 58
input [44, 53, 56, 1, 5

In [17]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, idx, targets=None):        
        logits = self.token_embedding_table(idx)
        if targets is None: # just get the logits
            loss = None
        else: # score the logits against targets
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # only focus on last time step
            logits = logits[:, -1, :]
            # convert to probabilities w/ softmax
            probs = F.softmax(logits, dim=-1)
            # sample next token from distribution
            idx_next = torch.multinomial(probs, num_samples=1)
            # append sample to running sequence
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
    
m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)
idx = torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, 
                       max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8235, grad_fn=<NllLossBackward0>)

yV!pOqR;tHZL,BBTsfez !mKq.FggmdzWAC3KR;.t?oxoHjr$BEaCIGV!mBzXefQioxoCXO.Ac;wrhc;comNYLPaMF
p:cwn'O.S


In [81]:
ff = xb[:2, :2].contiguous()
gg = yb[:2, :2].contiguous()
print(ff)
print(gg)

tensor([[24, 43],
        [44, 53]])
tensor([[43, 58],
        [53, 56]])


In [87]:
logits = m.token_embedding_table(ff)
print(logits.shape)
B, T, C = logits.shape
print(B,T,C)
logits = logits.view(B*T, C)
print(f"logits {logits.shape}")
print(gg.shape)
targets = gg.view(B*T)
print(f"targets {targets}")
loss = F.cross_entropy(logits, targets)
print(loss)

torch.Size([2, 2, 65])
2 2 65
logits torch.Size([4, 65])
torch.Size([2, 2])
targets tensor([43, 58, 53, 56])
tensor(4.7633, grad_fn=<NllLossBackward0>)


In [88]:
logits[:, 43]

tensor([ 0.6294,  0.6927,  1.0668, -1.3627], grad_fn=<SelectBackward0>)

In [26]:
r1, r2 = m(ff, gg)

RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

In [ ]:
# next up - Training!